In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import math

In [2]:
formulaCompressionTestHome = "C:\\Users\\jonat\\Documents\\Dataspread\\FormulaCompressionTest"

In [43]:
def getMiscDict(path):
    dirList = path.split('\\')
    syncType = dirList[len(dirList) - 7]
    memoryStorage = dirList[len(dirList) - 6]
    baseline = dirList[len(dirList) -5]
    if baseline.lower() == 'comp':
        baseline = 'TACO'
    elif baseline.lower() == 'async':
        baseline = 'Antifreeze'
    elif baseline.lower() == 'pgimpl':
        baseline = 'NoComp'
    testSpreadsheet = dirList[len(dirList) - 4]
    run = int(dirList[len(dirList) - 3].split('-')[1])
    rows = int(dirList[len(dirList) - 2].split('-')[1])
    return {'Sync/Async':syncType, 'Memory Storage':memoryStorage, 'Baseline':baseline,
            'Test Spreadsheet':testSpreadsheet, 'Run':run, 'Rows':rows}

def getMetrics(path):
    """
    Takes in a path string to a .stat file and returns a Pandas DataFrame of the file's contents
    """
    reportsFile = open(path, 'r')
    dataDict = {}
    dataDict = getMiscDict(path)
    for line in reportsFile:
        if ': ' in line:
            metric, value = line.split(": ")
            if metric not in dataDict:
                dataDict[metric] = []
            try:
                dataDict[metric].append(int(value))
            except ValueError:
                dataDict[metric].append(float(value))
            except:
                dataDict[metric].append(value)
    reportsFile.close
    
    return pd.DataFrame(dataDict)


def getAllMetrics(basePath, miscDict={'sync type':'', 'mem type':'', 'impl':'', 'test':'', 'run':0}):
    """
    Takes in a file path and searches all of its subdirectories for .stat files and returns all of
    their contents in a Pandas DataFrame
    """
    data = pd.DataFrame()
    for file in os.listdir(basePath):
        filePath = os.path.join(basePath, file)
        if os.path.isdir(filePath):
            data = data.append(getAllMetrics(filePath))
        elif file == "core.stat":
            data = data.append(getMetrics(filePath))
    return data


def plotData(data, 
             researchQuestion,
             plotFunc,
             relevantColumns,
             sheets=['Rate', 'RunningTotalFast', 'RunningTotalSlow'], 
             memStorage = ['DB', 'IN_MEM', 'MEM'],
             syncMethod=['async', 'sync'],
             save=False,
             plotHomePath='../plots/Dixin-Exp'):
    plt.rcParams.update({'font.size': 25, 'figure.figsize': (12,8), 'lines.linewidth':3})
    for sheet in sheets:
        for sync in syncMethod:
            for mem in memStorage:
                dataSlice = pd.DataFrame()
                try:
                    dataSlice = data.loc[sheet, mem, sync]
                except:
                    print('No data for (' + sheet + ', ' + mem + ', ' + sync + ')')
                
                if len(dataSlice > 0):
                    relevantData = pd.pivot_table(dataSlice, 
                                              index='Rows', 
                                              columns=dataSlice.index)[[relevantColumns]][relevantColumns]
                    relevantData = relevantData[relevantData.columns.tolist()[-2:] + relevantData.columns.tolist()[:-2]]
                    splitData = [relevantData[relevantData.index < 100000], relevantData[relevantData.index >= 100000]]
                    if len(splitData[0]) > 0:
                        plotFunc(splitData[0], researchQuestion, sheet, mem, sync, False, save)
                    if len(splitData[1]) > 0:
                        plotFunc(splitData[1], researchQuestion, sheet, mem, sync, True, save)
                        
def findYMax(data):
    digits = 0
    maxTime = 0
    for column in data.columns:
        maxTime = max(maxTime, data[column].max())
    digitTracker = maxTime
    while digitTracker > 10:
        digits += 1
        digitTracker = digitTracker // 10
    return math.ceil(maxTime / 10 ** digits) * 10**digits + 10 ** (digits - 1)
        
def checkForAntifreeze(df):
    return len(df['Antifreeze']) - df['Antifreeze'].isnull().sum()


def savePlot(plot, homePath, rq, sheet, mem, big):
    filePath = homePath+rq+'/'+sheet+'/'+mem+'/'
    Path(filePath).mkdir(parents=True, exist_ok=True)
    if big:
        filePath += sheet+'_'+mem+'_big.pdf'
    else:
        filePath += sheet+'_'+mem+'_small.pdf'
    plt.savefig(filePath, bbox_inches='tight')
    print('Plot saved to ' + filePath)

In [54]:
#RQ1
def plotTotalTime(plotData, researchQuestion, sheet, mem, sync, big, save):
    plt.figure()
    numAntifreeze = checkForAntifreeze(plotData)
    if numAntifreeze == 0:
        plotData.drop(columns='Antifreeze', inplace=True)
    elif numAntifreeze < 5:
        'label last point'
    
    if big:
        for column in plotData.columns:
            plotData[column] = plotData[column] / 1000
    lineplt = sns.lineplot(markers=True, markersize=20, ci=None, dashes=False, alpha=.5, data=plotData)
    lineplt.yaxis.grid()
    lineplt.legend(markerscale=3)
    lineplt.set_xticklabels(['{:,.0f}'.format(x) + 'K' for x in lineplt.get_xticks()/1000])
    lineplt.set_ybound(upper=findYMax(plotData))
    if 0 < numAntifreeze < 5:
        row = plotData.iloc[numAntifreeze - 1]
        lineplt.text(x=int(row.name) + lineplt.get_xlim()[1] * .015, 
                       y=plotData.loc[row.name, 'Antifreeze'] + ((lineplt.get_ylim()[1] - lineplt.get_ylim()[0]) * .03),
                       fontsize=20,
                       s='Antifreeze does not finish\nafter this point')
    
    handles, labels = batchPlot.get_legend_handles_labels()
    leg = batchPlot.legend(
        bbox_to_anchor=(.65, .2, 1, 1),
        loc="lower left",
        handles=handles[0:], 
        labels=labels[0:],
        markerscale=3
        )
    timeUnit = '(ms)'
    if big:
        timeUnit = '(s)'
    lineplt.set_ylabel('Time to Return Control to Users ' + timeUnit)
    lineplt.set_xlabel('Number of Rows ')
    if save: 
        savePlot(lineplt, '../plots/Dixin-Exp/', researchQuestion, sheet, mem, big)
        plt.close()
        
        
def plotBars(testData, numRows, big, save=False):
    relevantData = testData[testData['Rows'] == numRows]
    try:
        relevantData = relevantData.loc[:, 'MEM', 'async']
    except:
        relevantData = relevantData.loc[:, 'IN_MEM', 'async']
        #relevantData = relevantData[relevantData.columns.tolist()[-2:] + relevantData.columns.tolist()[:-2]]
    relevantData = relevantData[['Total time to update cells (ms)']]
    relevantData = relevantData.reset_index(level=['Test Spreadsheet', 'Baseline']).pivot(index='Test Spreadsheet', columns='Baseline', values='Total time to update cells (ms)')
    relevantData = relevantData[relevantData.columns.tolist()[-2:] + relevantData.columns.tolist()[:-2]]
    ax = relevantData.plot.bar()
    yMax = 200
    ax.set_ylabel('Time of Returning Control to Users (ms)')
    ax.set_xlabel('')
    ax.set_xticklabels(['Rate', 'Fast', 'Slow'])
    ax.set_ybound(upper=yMax)
    ax.yaxis.grid()
    for bar in ax.patches:
        if bar.get_height() > yMax:
            ax.annotate('{:.1f}'.format(bar.get_height() / 1000) + '\nsec',
                           (bar.get_x() + bar.get_width() / 2, yMax * .95), ha='center', va='center',
                           size=23, xytext=(0, -8),
                           textcoords='offset points')
    handles, labels = ax.get_legend_handles_labels()
    leg = ax.legend(
        bbox_to_anchor=(.65, .2, 1, 1),
        loc="lower left",
        handles=handles[0:], 
        labels=labels[0:],
        markerscale=3
        )
    
    if save:
        filePath = '../plots/Dixin-Exp/rq1/MemBarPlots/'
        Path(filePath).mkdir(parents=True, exist_ok=True)
        if big:
            plt.savefig(filePath + 'mem_' + 'big.pdf', bbox_inches="tight")
        else:
            plt.savefig(filePath + 'mem_' + 'small.pdf', bbox_inches="tight")
        plt.close()
        
        
formulaCompressionTestHome = "C:\\Users\\jonat\\Documents\\Dataspread\\FormulaCompressionTest"
testPath = formulaCompressionTestHome + "\\experiments_data\\rq1\\report"
pd.set_option('max_rows', 115)
testData = getAllMetrics(testPath)
testData = testData.groupby(['Test Spreadsheet', 'Memory Storage', 'Sync/Async', 'Baseline', 'Rows']).mean()
testData = testData.reset_index(level='Rows')
#display(testData)
plotData(testData, 'rq1', plotTotalTime, 'Total time to update cells (ms)', memStorage=['DB'], save=True)

plotBars(testData, 10000, False, save=True)
plotBars(testData, 500000, True, save=True)

<ipython-input-54-1e9e0d7f73d8>:16: UserWarning: FixedFormatter should only be used together with FixedLocator
  lineplt.set_xticklabels(['{:,.0f}'.format(x) + 'K' for x in lineplt.get_xticks()/1000])
C:\Users\jonat\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-54-1e9e0d7f73d8>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plotData[column] = plotData[column] / 1000
<ipython-input-54-1e9e0d7f73d8>:16: UserWarning: FixedFormatter should only be used together with FixedLoc

Plot saved to ../plots/Dixin-Exp/rq1/Rate/DB/Rate_DB_small.pdf
Plot saved to ../plots/Dixin-Exp/rq1/Rate/DB/Rate_DB_big.pdf
No data for (Rate, DB, sync)


<ipython-input-54-1e9e0d7f73d8>:16: UserWarning: FixedFormatter should only be used together with FixedLocator
  lineplt.set_xticklabels(['{:,.0f}'.format(x) + 'K' for x in lineplt.get_xticks()/1000])
C:\Users\jonat\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-54-1e9e0d7f73d8>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plotData[column] = plotData[column] / 1000
<ipython-input-54-1e9e0d7f73d8>:16: UserWarning: FixedFormatter should only be used together with FixedLoc

Plot saved to ../plots/Dixin-Exp/rq1/RunningTotalFast/DB/RunningTotalFast_DB_small.pdf
Plot saved to ../plots/Dixin-Exp/rq1/RunningTotalFast/DB/RunningTotalFast_DB_big.pdf
No data for (RunningTotalFast, DB, sync)


<ipython-input-54-1e9e0d7f73d8>:16: UserWarning: FixedFormatter should only be used together with FixedLocator
  lineplt.set_xticklabels(['{:,.0f}'.format(x) + 'K' for x in lineplt.get_xticks()/1000])
C:\Users\jonat\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-54-1e9e0d7f73d8>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plotData[column] = plotData[column] / 1000
<ipython-input-54-1e9e0d7f73d8>:16: UserWarning: FixedFormatter should only be used together with FixedLoc

Plot saved to ../plots/Dixin-Exp/rq1/RunningTotalSlow/DB/RunningTotalSlow_DB_small.pdf
Plot saved to ../plots/Dixin-Exp/rq1/RunningTotalSlow/DB/RunningTotalSlow_DB_big.pdf
No data for (RunningTotalSlow, DB, sync)


In [48]:
#RQ2
def plotBatchLoadingTime(plotData, researchQuestion, sheet, mem, sync, big, save):
    plt.figure()
    yMax = findYMax(plotData[['NoComp', 'TACO']])
    numAntifreeze = checkForAntifreeze(plotData)
    if numAntifreeze == 0:
        plotData.drop(columns='Antifreeze', inplace=True)
    cappedData = plotData.copy()
    for column in plotData.columns:
        cappedData[column] = plotData[column].map(lambda x: min(x, yMax * .95))
    if big:
        for column in plotData.columns:
            plotData[column] = plotData[column] / 1000
    batchPlot = sns.lineplot(markers=True, markersize=20, ci=None, dashes=False, alpha=.5, color="black", data=cappedData)
    batchPlot.set_xticklabels(['{:,.0f}'.format(x) + 'K' for x in batchPlot.get_xticks()/1000])
    if 0 < numAntifreeze < 5:
        row = plotData.iloc[numAntifreeze - 1]
        batchPlot.text(x=int(row.name) + batchPlot.get_xlim()[1] * .015, 
                       y=cappedData.loc[row.name, column] - (batchPlot.get_ylim()[1] + batchPlot.get_ylim()[0]) * .045,
                       fontsize=20,
                       s='Antifreeze does not finish\nafter this point')
    batchPlot.yaxis.grid()
    batchPlot.set_ybound(upper=yMax)
    xAdjust = - max(plotData.index.values) * .03
    yAdjust = yMax * .1
    xMax = max(plotData.index.values)
    for column in plotData.columns:
        for i, row in plotData[plotData[column] > cappedData[column]].iterrows():
            batchPlot.text(x=int(row.name) + xAdjust, 
                           y=cappedData.loc[row.name, column] - yAdjust, 
                           s='{:.1f}'.format(row[column] / 60000) + ' min')
    batchPlot.legend(markerscale=3)
    timeUnit = '(ms)'
    handles, labels = batchPlot.get_legend_handles_labels()
    leg = batchPlot.legend(
        bbox_to_anchor=(.65, .2, 1, 1),
        loc="lower left",
        handles=handles[0:], 
        labels=labels[0:],
        markerscale=3
        )
    if big:
        timeUnit = '(s)'
    batchPlot.set_ylabel('Time of Building the Formula Graph ' + timeUnit)
    batchPlot.set_xlabel('Number of Rows')
    if save: 
        savePlot(batchPlot, '../plots/Dixin-Exp/', researchQuestion, sheet, mem, big)
        plt.close()
        
        
        
        
        
formulaCompressionTestHome = "C:\\Users\\jonat\\Documents\\Dataspread\\FormulaCompressionTest"
testPath = formulaCompressionTestHome + "\\experiments_data\\rq2\\report"
pd.set_option('max_rows', 115)
testData = getAllMetrics(testPath)
testData = testData.groupby(['Test Spreadsheet', 'Memory Storage', 'Sync/Async', 'Baseline', 'Rows']).mean()
testData = testData.reset_index(level='Rows')
#display(testData)

plotData(testData, 'rq2', plotBatchLoadingTime, 'Total time of adding the batch (ms)', memStorage=['DB'], save=True)



<ipython-input-48-5895d6435468>:15: UserWarning: FixedFormatter should only be used together with FixedLocator
  batchPlot.set_xticklabels(['{:,.0f}'.format(x) + 'K' for x in batchPlot.get_xticks()/1000])
C:\Users\jonat\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-48-5895d6435468>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plotData[column] = plotData[column] / 1000
<ipython-input-48-5895d6435468>:15: UserWarning: FixedFormatter should only be used together with Fixe

Plot saved to ../plots/Dixin-Exp/rq2/Rate/DB/Rate_DB_small.pdf
Plot saved to ../plots/Dixin-Exp/rq2/Rate/DB/Rate_DB_big.pdf
No data for (Rate, DB, sync)


<ipython-input-48-5895d6435468>:15: UserWarning: FixedFormatter should only be used together with FixedLocator
  batchPlot.set_xticklabels(['{:,.0f}'.format(x) + 'K' for x in batchPlot.get_xticks()/1000])
C:\Users\jonat\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-48-5895d6435468>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plotData[column] = plotData[column] / 1000
<ipython-input-48-5895d6435468>:15: UserWarning: FixedFormatter should only be used together with Fixe

Plot saved to ../plots/Dixin-Exp/rq2/RunningTotalFast/DB/RunningTotalFast_DB_small.pdf
Plot saved to ../plots/Dixin-Exp/rq2/RunningTotalFast/DB/RunningTotalFast_DB_big.pdf
No data for (RunningTotalFast, DB, sync)


<ipython-input-48-5895d6435468>:15: UserWarning: FixedFormatter should only be used together with FixedLocator
  batchPlot.set_xticklabels(['{:,.0f}'.format(x) + 'K' for x in batchPlot.get_xticks()/1000])
C:\Users\jonat\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-48-5895d6435468>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plotData[column] = plotData[column] / 1000
<ipython-input-48-5895d6435468>:15: UserWarning: FixedFormatter should only be used together with Fixe

Plot saved to ../plots/Dixin-Exp/rq2/RunningTotalSlow/DB/RunningTotalSlow_DB_small.pdf
Plot saved to ../plots/Dixin-Exp/rq2/RunningTotalSlow/DB/RunningTotalSlow_DB_big.pdf
No data for (RunningTotalSlow, DB, sync)


In [50]:
#RQ3
def plotRQ3(plotData, researchQuestion, sheet, mem, sync, big, save):
    plt.figure()
    numAntifreeze = checkForAntifreeze(plotData)
    if numAntifreeze == 0:
        plotData.drop(columns='Antifreeze', inplace=True)
    elif numAntifreeze < 5:
        'label last point'
    cappedData = plotData.copy()
    for column in plotData.columns:
        plotData[column] = plotData[column] / 1000
    yMax = 100
    for column in plotData.columns:
        cappedData[column] = plotData[column].map(lambda x: min(x, yMax * .95))
    batchPlot = sns.lineplot(markers=True, markersize=20, ci=None, dashes=False, alpha=.5, color="black", data=cappedData)
    batchPlot.set_xticklabels(['{:,.0f}'.format(x) + 'K' for x in batchPlot.get_xticks()/1000])
    batchPlot.yaxis.grid()
    xAdjust = - max(plotData.index.values) * .03
    yAdjust = yMax * .1
    xMax = max(plotData.index.values)
    for column in plotData.columns:
        for i, row in plotData[plotData[column] > cappedData[column]].iterrows():
            xMax = max(plotData.index.values) * 1.14
            batchPlot.text(x=int(row.name) + xAdjust, 
                           y=cappedData.loc[row.name, column] - yAdjust, 
                           s='{:.1f}'.format(row[column] / 60) + ' min')
    batchPlot.set_ybound(upper=yMax)
    batchPlot.set_xbound(upper=xMax)
    batchPlot.legend(markerscale=3)
    timeUnit = '(s)'
    batchPlot.set_ylabel('Time of Modifying the Formula Graph ' + timeUnit)
    batchPlot.set_xlabel('Number of Rows Modified')
    if save: 
        savePlot(batchPlot, '../plots/Dixin-Exp/', researchQuestion, sheet, mem, big)
        plt.close()


testPath = formulaCompressionTestHome + "\\experiments_data\\rq3\\report"
pd.set_option('max_rows', 115)
testData = getAllMetrics(testPath)
testData = testData.groupby(['Test Spreadsheet', 'Memory Storage', 'Sync/Async', 'Baseline', 'Rows']).mean()
testData = testData.reset_index(level='Rows')

plotData(testData, 'rq3', plotRQ3, 'Total time of refreshing the cache (ms)', sheets=['RefreshCache'], memStorage=['DB', 'MEM'], save=True)

<ipython-input-50-ac30f2202d1b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plotData[column] = plotData[column] / 1000
<ipython-input-50-ac30f2202d1b>:16: UserWarning: FixedFormatter should only be used together with FixedLocator
  batchPlot.set_xticklabels(['{:,.0f}'.format(x) + 'K' for x in batchPlot.get_xticks()/1000])
C:\Users\jonat\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-50-ac30f2202d1b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

Plot saved to ../plots/Dixin-Exp/rq3/RefreshCache/DB/RefreshCache_DB_small.pdf
Plot saved to ../plots/Dixin-Exp/rq3/RefreshCache/DB/RefreshCache_DB_big.pdf


<ipython-input-50-ac30f2202d1b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plotData[column] = plotData[column] / 1000
<ipython-input-50-ac30f2202d1b>:16: UserWarning: FixedFormatter should only be used together with FixedLocator
  batchPlot.set_xticklabels(['{:,.0f}'.format(x) + 'K' for x in batchPlot.get_xticks()/1000])
C:\Users\jonat\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-50-ac30f2202d1b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

Plot saved to ../plots/Dixin-Exp/rq3/RefreshCache/MEM/RefreshCache_MEM_small.pdf
Plot saved to ../plots/Dixin-Exp/rq3/RefreshCache/MEM/RefreshCache_MEM_big.pdf
No data for (RefreshCache, DB, sync)
No data for (RefreshCache, MEM, sync)
